In [6]:
import os
import re
import io
import requests
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import warnings

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Recurrent Neural Networks
* Recurrent networks share their weights over time, thus "recurrent"
* This allows them to have a kind of "memory" about what happened earlier in the sequence.

<img src="../pics/RNN-unrolled.png">

* Here we create an RNN composed of a layer of five recurrent neurons using ReLU for our activation function. 
* In this simple case, we are assuming that the RNN runs over only two-time steps, taking input vectors of size 3 at each time step. 

In [2]:
n_inputs = 3
n_neurons = 5

X1 = tf.placeholder(tf.float32, [None, n_inputs])
X2 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.get_variable("Wx", shape=[n_inputs,n_neurons], dtype=tf.float32, initializer=None, regularizer=None, trainable=True, collections=None)

Wy = tf.get_variable("Wy", shape=[n_neurons,n_neurons], dtype=tf.float32, initializer=None, regularizer=None, trainable=True, collections=None)

b = tf.get_variable("b", shape=[1,n_neurons], dtype=tf.float32, initializer=None, regularizer=None, trainable=True, collections=None)

Y1 = tf.nn.relu(tf.matmul(X1, Wx) + b)
Y2 = tf.nn.relu(tf.matmul(Y1, Wy) + tf.matmul(X2, Wx) + b)

init_op = tf.global_variables_initializer()

* This network looks much like a two-layer feedforward neural network, but both layers share the same weights and bias vectors. 
* Also, we feed inputs at each layer and receive outputs from each layer.

In [3]:
# Mini-batch: instance 0,instance 1,instance 2,instance 3
X1_batch = np.array([[0, 2, 3], [2, 8, 9], [5, 3, 8], [3, 2, 9]]) # t = 0
X2_batch = np.array([[5, 6, 8], [1, 0, 0], [8, 2, 0], [2, 3, 6]]) # t = 1

* These mini-batches contain four instances, each with an input sequence composed of exactly two inputs. 
* At the end, Y1_val and Y2_val contain the outputs of the network at both time steps for all neurons and all instances in the mini-batch.

In [4]:
with tf.Session() as sess:
	init_op.run()
	Y1_val, Y2_val = sess.run([Y1, Y2], feed_dict={X1: X1_batch, X2: X2_batch})

print(Y1_val) # output at t = 0
print()
print(Y2_val) # output at t = 1

[[0.         0.         0.51920843 0.         0.8595352 ]
 [0.         0.         1.2733305  0.         0.7417767 ]
 [0.         0.         0.         0.         4.500122  ]
 [0.         0.         0.         0.         4.3668284 ]]

[[0.         0.         0.         0.         3.0322785 ]
 [0.         0.         0.         0.21159875 1.2591668 ]
 [0.         0.         0.         6.5043755  5.023193  ]
 [0.         0.         0.         0.         3.5113583 ]]


### Using contrib

In [7]:
tf.reset_default_graph()
n_inputs = 3
n_neurons = 5
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
seq_length = tf.placeholder(tf.int32, [None])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

# note that here instance one is not as long as the other sequences
# so we padded it with a zero vector
X_batch = np.array([
                   [[0, 2, 3], [2, 8, 9]], # instance 0
                   [[5, 6, 8], [0, 0, 0]], # instance 1 (padded with a zero vector)
                   [[6, 7, 8], [6, 5, 4]], # instance 2
                   [[8, 2, 0], [2, 3, 6]], # instance 3
                  ])
seq_length_batch = np.array([3, 4, 3, 5])
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
        init_op.run()
        outputs_val, states_val = sess.run([output_seqs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

print(outputs_val)

[[[-0.03298712  0.3656067   0.7263861  -0.39105207 -0.8217619 ]
  [ 0.3379907   0.803342    0.9320661  -0.9895262  -0.9953831 ]]

 [[ 0.96835923 -0.92017645 -0.4448912  -0.9978729  -0.86391944]
  [ 0.7141644  -0.22028413  0.58800185 -0.34547463 -0.5787524 ]]

 [[ 0.9746464  -0.92241496 -0.6864854  -0.9993912  -0.51832616]
  [ 0.99151057 -0.9645286  -0.8491261  -0.9991241   0.6904297 ]]

 [[ 0.99596894 -0.9986037  -0.9997294  -0.9991608   0.9971175 ]
  [ 0.82204396 -0.8652973   0.36278024 -0.7410866  -0.9432218 ]]]
